# Redshift fitting for CANUCS test sources

In [ ]:
#import specutils
import specutils
from specutils import Spectrum1D
from specutils.fitting import fit_generic_continuum
from specutils.analysis import correlation
from specutils.manipulation import LinearInterpolatedResampler

#import jdaviz
import jdaviz
from jdaviz import SpecViz

#import matplotlib
from matplotlib import pyplot as plt

#import astropy
import astropy
from astropy.io import fits,ascii
from astropy import units as u
from astropy.table import QTable
from astropy.modeling.models import Linear1D, Polynomial1D, Chebyshev1D
from astropy.nddata import StdDevUncertainty

#import numpy
import numpy as np

In [ ]:
#customization of matplotlib style
plt.rcParams["figure.figsize"] = (10,5)
params={'legend.fontsize':'18','axes.labelsize':'18',
        'axes.titlesize':'18','xtick.labelsize':'18',
        'ytick.labelsize':'18','lines.linewidth':2,
        'axes.linewidth':2,'animation.html': 'html5',
        'figure.figsize':(8,6)}
plt.rcParams.update(params)
plt.rcParams.update({'figure.max_open_warning': 0})

Versions

In [ ]:
print("Specutils:", specutils.__version__)
print("Astropy:", astropy.__version__)
print("Jdaviz:", jdaviz.__version__)

## Get a template first since I need this only once

In [ ]:
spec_unit = u.erg / (u.s * u.cm**2 * u.AA)

template_file = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/redshift_crosscorr/00076.dat'

template = ascii.read(template_file)
factor = 2.E-5 * spec_unit # normalize template to a sensible range
template = Spectrum1D(spectral_axis=template['col1']/1E4*u.um, 
                      flux=template['col2']*factor)

plt.plot(template.spectral_axis, template.flux)
plt.xlabel('wavelength (AA)')
plt.ylabel('flux (arbitrary)')
plt.xlim(0.3,0.8)
plt.show()

In [ ]:
#cut to useful range - template and obs MUST overlap, so we go to 1.1um
con_tmp = (template.spectral_axis.value > 0.3) & (template.spectral_axis.value < 1.1)
template_cut = Spectrum1D(spectral_axis=template.spectral_axis[con_tmp],flux=template.flux[con_tmp])

plt.plot(template_cut.spectral_axis, template_cut.flux)
plt.xlabel('wavelength (AA)')
plt.ylabel('flux (arbitrary)')
plt.show()

## Subtract continuum and replot

In [ ]:
#subtract continuum
mask_temp = ((template_cut.spectral_axis.value > 0.31) & (template_cut.spectral_axis.value < 0.37) |
             (template_cut.spectral_axis.value > 0.40) & (template_cut.spectral_axis.value < 0.47) |
             (template_cut.spectral_axis.value > 0.52) & (template_cut.spectral_axis.value < 0.62) |
             (template_cut.spectral_axis.value > 0.70) & (template_cut.spectral_axis.value < 1.05))

template_forcont = Spectrum1D(spectral_axis=template_cut.spectral_axis[mask_temp],flux=template_cut.flux[mask_temp])

fit_temp = fit_generic_continuum(template_forcont, model=Polynomial1D(5))
cont_temp = fit_temp(template_cut.spectral_axis)

plt.plot(template_cut.spectral_axis, template_cut.flux)
plt.plot(template_cut.spectral_axis, cont_temp)
plt.xlabel('wavelength (um)')
plt.ylabel('flux (arbitrary)')
plt.show()

In [ ]:
template_sub = template_cut - cont_temp

plt.plot(template_sub.spectral_axis, template_sub.flux)
plt.xlabel('wavelength (um)')
plt.ylabel('flux (arbitrary)')
plt.show()

print(spec1d_sub.spectral_axis_unit, template_sub.spectral_axis_unit)

## Read spectrum

In [ ]:
galid = 19
filespec = "./jaguar_grism_spectra_v2/canucs_basic_WFSSC_{}_combine_1d.fits".format(galid)

spec = fits.open(filespec)
spec.info()

spec[1].header

In [ ]:
wave = spec[1].data['wavelength']
flux = spec[1].data['flux'] * 1.E5

dataspec = QTable([wave*u.um, flux*spec_unit, flux*0.1*spec_unit],
                   names=('wavelength','flux','unc'))

dataspec.sort('wavelength')

spec1d = Spectrum1D(spectral_axis = dataspec['wavelength'], 
                    flux = dataspec['flux'], 
                    uncertainty=StdDevUncertainty(dataspec['unc']))

Maybe I need a spectrum with no holes? The delta_lambda is ~0.004.

In [ ]:
newwave = np.arange(1.050,2.23,0.004) * u.um

linear = LinearInterpolatedResampler()
spec1d_full = linear(spec1d, newwave)

In [ ]:
plt.plot(spec1d_full.spectral_axis, spec1d_full.flux)
plt.xlabel('wavelength (um)')
plt.ylabel('flux (arbitrary)')
plt.show()

## Fit continuum

In [ ]:
mask = ((spec1d_full.spectral_axis > 1.1 * u.um) & (spec1d_full.spectral_axis < 1.2 * u.um) | 
        (spec1d_full.spectral_axis > 1.45 * u.um) & (spec1d_full.spectral_axis < 1.6 * u.um) |
        (spec1d_full.spectral_axis > 2.0 * u.um) & (spec1d_full.spectral_axis < 2.2 * u.um))

spec1d_cont = Spectrum1D(spectral_axis=spec1d_full.spectral_axis[mask],flux=spec1d_full.flux[mask])

In [ ]:
cont_spec1d = fit_generic_continuum(spec1d_cont, model=Polynomial1D(3))
continuum = cont_spec1d(spec1d.spectral_axis)

plt.plot(spec1d.spectral_axis, spec1d.flux)
plt.plot(spec1d.spectral_axis, continuum)
plt.xlabel('wavelength (um)')
plt.ylabel('flux (arbitrary)')
plt.show()

## Subtract continuum and replot

In [ ]:
spec1d_sub = spec1d - continuum

plt.plot(spec1d_sub.spectral_axis, spec1d_sub.flux)
plt.xlabel('wavelength (um)')
plt.ylabel('flux (arbitrary)')
plt.show()

## Cross correlation to measure redshift

In [ ]:
corr, lag = correlation.template_correlate(spec1d_sub, template_sub, lag_units=u.dimensionless_unscaled)

print(corr,lag)

plt.plot(lag, corr)
plt.xlabel(lag.unit)

In [ ]:
# Redshift based on maximum
index_peak = np.argmax(corr)

z = lag[index_peak]

print("Redshift from peak maximum: ", z)


## Visual inspect the fit

In [ ]:
template_z = Spectrum1D(spectral_axis=template_sub.spectral_axis * (1. + z), flux=template_sub.flux/1000)

plt.plot(spec1d_sub.spectral_axis, spec1d_sub.flux, label='Data')
plt.plot(template_z.spectral_axis, template_z.flux, label='Template at z', alpha=0.3)
plt.xlabel('wavelength (um)')
plt.ylabel('flux (arbitrary)')
plt.show()

In [ ]:
#redshift based on visual inspection
viz = SpecViz()
viz.app

In [ ]:
viz.load_spectrum(spec1d_sub,"CANUCS {}".format(galid))